Helm
----

![](https://helm.sh/img/chart-illustration.png)

Quelle: https://helm.sh/
- - - 

[Helm](https://helm.sh/) ist ein Werkzeug zur Verwaltung von Kubernetes-Charts. 

[Kubernetes-Charts](https://github.com/helm/charts) sind Pakete vorkonfigurierter Kubernetes-Ressourcen, welche wie Gesamthaft mittels eines Befehles installiert werden können

Der [Helm Hub](https://hub.helm.sh) enthält Dokumentationen und Konfigurationenen für die von Helm gehosteten Charts.

Helm Einsatzgebiete:
* Suchen und verwenden Sie beliebte Software, die als Kubernetes-Charts verpackt ist
* Teilen Sie Ihre eigenen Anwendungen als Kubernetes-Charts
* Erstellen Sie reproduzierbare Builds Ihrer Kubernetes-Anwendungen
* Verwalten Sie Ihre Kubernetes-Manifestdateien auf intelligente Weise
* Verwalten Sie Releases von Helm-Paketen

***
### Installation

Zuerst müssen wir `helm` Installieren. Ab Version 3.x braucht es dazu nur noch das CLI `helm` und keine Services mehr auf dem Kubernetes Cluster.

Nach der Installation muss mindestens ein Repository hinzufügt werden, damit Charts gefunden werden.


In [ ]:
! curl https://get.helm.sh/helm-v3.1.0-linux-amd64.tar.gz | tar xzf -
! mv linux-amd64/helm .
! ./helm repo add stable https://kubernetes-charts.storage.googleapis.com/

***
### Services starten

Nun sind wir bereit und können Services, mittels [Kubernetes-Charts](https://github.com/helm/charts), installieren.

Beginnen wollen wir mit [Jenkins](https://jenkins.io/):

In [ ]:
! ./helm install myjenkins stable/jenkins --set master.serviceType=LoadBalancer

Im Gegensatz zum Recht einfachen Beispiel [Jenkins mit Blueocean](https://github.com/mc-b/duk/tree/master/devops#jenkins-mit-blueocean) wurden einiges mehr an Ressourcen erstellt u.a.:

* ConfigMap für die Konfiguration
* PersistenVolumeClaim zur Ablage der Daten
* Role und Service Account für Interation Agent - Jenkins?

Ausserdem wurde Password generiert. Diese und den UI-Port können wir wie folgt abfragen:

In [ ]:
! echo "User    : admin"
! echo "Password: "$(kubectl get secret --namespace default myjenkins -o jsonpath="{.data.jenkins-admin-password}" | base64 --decode)
! echo "UI      : "$(kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e 's/https:/http:/' -e "s/6443/$(kubectl get service myjenkins -o=jsonpath='{ .spec.ports[0].nodePort }')/")

Wie bei [Jenkins mit Blueocean](https://github.com/mc-b/duk/tree/master/devops#jenkins-mit-blueocean) können wir mittels "neues Projekt" -> "Multibranch Pipeline" und Angabe eines Git Repositories, z.B. [https://github.com/mc-b/simple-java-maven-app](https://github.com/mc-b/simple-java-maven-app) im User Interface einen Job anlegen. Im [simple-java-maven-app](https://github.com/mc-b/simple-java-maven-app) Projekt steht auch eine [Jenkinsfile-k8s](https://github.com/mc-b/simple-java-maven-app/blob/master/Jenkinsfile-k8s) Datei zur Verfügung, welche mit dem [Kubernetes Jenkins plugin](https://plugins.jenkins.io/kubernetes/) zusammenspielt.

Um den Job zu bauen müssen wir zuerst Secret File (kubeconfig file) von Kubernetes in den Einstellungen von Jenkins eintragen.

Welche Schritte dazu nötig sind und alle weiteren Informationen stehen im [Kubernetes Jenkins plugin](https://plugins.jenkins.io/kubernetes/).


- - - 
Wenn wir Jenkins nicht mehr brauchen, können wir den Service wieder löschen:


In [ ]:
! ./helm uninstall myjenkins